In [14]:
import math
from datetime import time
from numpy import mean, std
from utils import (
    to_datetime,
    get_lective_data,
    get_lective_dates,
    next_days,
    find_hour_day,
    plot_and_save
)

In [11]:
# be careful with the holidays
# at the method is_lective_day()
# 2-21: Carnaval

PATH = "2020_2to3.csv"
labels, data = get_lective_data(PATH)
dates = get_lective_dates(data[0][0], data[-1][0])

keys = [
    'eCO2 in ppm (AMS CCS811 - eCO2)',
    'TVOC in ppb (AMS CCS811 - TVOC)',
    'PM 1 in ug/m3 (PMS5003_AVG-PM1)',
    'PM 10 in ug/m3 (PMS5003_AVG-PM10)',
    'PM 2.5 in ug/m3 (PMS5003_AVG-PM2.5)'
]

standards = [
    1000,
    150, # 0.3/2 * 1000 -> very good limit
    None,
    50,
    25
]


names = [
    'eCO2',
    'TVOC',
    'PM1',
    'PM10',
    'PM25'
]


horaris = [ (time(s_h, s_m), time(e_h, e_m)) for s_h, s_m, e_h, e_m in 
    [
        ( 8, 0,  8,45),
        ( 8,45,  9,30),
        ( 9,30, 10,15),
        (10,45, 11,30),
        (11,30, 12,15),
        (12,15, 13, 0),
        (13,10, 13,55),
        (13,55, 14,40)
    ]
]

dies_lectius = list(range(5))


In [12]:
# to calculate the means and the stdv of every day
means = dict.fromkeys(labels[1:])
stdev = dict.fromkeys(labels[1:])


for label in labels[1:]:
    means[label] = {}
    stdev[label] = {}

for key in means.keys():
    for date in dates:
        means[key][date] = []
        stdev[key][date] = []


for d in data:
    date = d[0][:10]
    for i, label in enumerate(means.keys()):
        if not math.isnan(d[i+1]):
            means[label][date].append(d[i+1])

for key in means.keys():
    for date in dates:
        m = round(mean(means[key][date]),1)
        d = round(std(means[key][date]),1)
        means[key][date] = m
        stdev[key][date] = d
        
for i, key in enumerate(keys):
    X = []
    Y = []
    std = []
    standard = standards[i]
    for date in means[key].keys():
        X.append(date[6:])
        Y.append(means[key][date])
        std.append(stdev[key][date])
    filename = 'DAY ' + names[i] + ' (with standard)'
    plot_and_save(X, Y, std, 'Date', key, filename, standard)

<Figure size 1080x504 with 0 Axes>

In [17]:
# To calculate the means for every 8 days

means_8days = dict.fromkeys(means.keys())
for key in means_8days.keys():
    means_8days[key] = {}

now = "2020-02-05"
end = "2020-03-13"
f = "%Y-%m-%d"
gen8 = next_days(to_datetime(now, f), 8)

tmp_dates = []
while now <= end:
    tmp_dates.append(now)
    now = next(gen8).strftime(f)

for key in means_8days.keys():
    for date in tmp_dates:
        means_8days[key][date] = []

i = 0
j = 0
#print(tmp_dates, tmp)
for label in means.keys():
    for date in means[label].keys():
        if i+1 < len(tmp_dates) and date >= tmp_dates[i+1]:
            i += 1
        means_8days[label][tmp_dates[i]].append(means[label][date])
    i = 0

for label in means_8days.keys():
    for date in tmp_dates:
        means_8days[label][date] = round(mean(means_8days[label][date]), 1)

means_8days

{'humidity in % (SHT31 - Humidity)': {'2020-02-05': 46.0,
  '2020-02-13': 49.0,
  '2020-02-21': 44.3,
  '2020-02-29': 38.7,
  '2020-03-08': 45.0},
 'air temperature in ºC (SHT31 - Temperature)': {'2020-02-05': 25.4,
  '2020-02-13': 24.7,
  '2020-02-21': 24.8,
  '2020-02-29': 25.6,
  '2020-03-08': 25.2},
 'barometric pressure in K Pa (MPL3115A2 - Barometric Pressure)': {'2020-02-05': 102.4,
  '2020-02-13': 102.5,
  '2020-02-21': 101.9,
  '2020-02-29': 100.5,
  '2020-03-08': 102.0},
 'battery in % (Battery SCK 1.1)': {'2020-02-05': 99.0,
  '2020-02-13': 99.0,
  '2020-02-21': 99.0,
  '2020-02-29': 99.0,
  '2020-03-08': 99.0},
 'eCO2 in ppm (AMS CCS811 - eCO2)': {'2020-02-05': 536.9,
  '2020-02-13': 969.8,
  '2020-02-21': 1015.3,
  '2020-02-29': 1296.1,
  '2020-03-08': 1337.0},
 'TVOC in ppb (AMS CCS811 - TVOC)': {'2020-02-05': 20.5,
  '2020-02-13': 95.1,
  '2020-02-21': 97.0,
  '2020-02-29': 156.7,
  '2020-03-08': 169.4},
 'light in Lux (BH1730FVC)': {'2020-02-05': 41.6,
  '2020-02-13': 4

In [18]:
time_table = dict.fromkeys(keys)

for key in keys:
    time_table[key] = []
    for i in range(len(horaris)):
        time_table[key].append([])
        for j in range(len(dies_lectius)):
            time_table[key][i].append([])
            
# now we have something like:
#{'eCO2 in ppm (AMS CCS811 - eCO2)': 
# [[[], [], [], [], []],
#  [[], [], [], [], []],
#  [[], [], [], [], []],
#  [[], [], [], [], []],
#  [[], [], [], [], []],
#  [[], [], [], [], []],
#  [[], [], [], [], []],
#  [[], [], [], [], []]],
# 'X':[...], 'Y':[...]}

for row in data:
    pack = find_hour_day(row[0], horaris, dies_lectius)
    # print('date:', row[0], 'pack', pack)
    if pack is None:
        continue
    hora, dia = pack
    for i, number in enumerate([5, 6, 8, 10, 11]):
        if not math.isnan(row[number]):
            time_table[keys[i]][hora][dia].append(row[number]) # humidity
            
# we have filled the data to the time_table

for key in keys:
    for i in range(len(horaris)):
        for j in range(len(dies_lectius)):
            time_table[key][i][j] = round(mean(time_table[key][i][j]), 1)

time_table

{'eCO2 in ppm (AMS CCS811 - eCO2)': [[1062.9, 1190.4, 1525.1, 1258.3, 1225.3],
  [1106.4, 1113.3, 1409.5, 1282.1, 1265.9],
  [1007.1, 1072.1, 1249.0, 1177.3, 1101.0],
  [933.8, 1034.9, 979.4, 1051.7, 997.5],
  [891.8, 895.5, 901.7, 1122.7, 922.2],
  [860.1, 772.2, 867.3, 982.0, 807.8],
  [883.7, 757.5, 934.8, 864.7, 960.4],
  [715.6, 765.4, 899.3, 786.5, 856.9]],
 'TVOC in ppb (AMS CCS811 - TVOC)': [[105.8, 143.1, 288.9, 196.9, 130.5],
  [113.8, 124.1, 210.5, 172.9, 149.3],
  [99.8, 108.5, 143.6, 141.3, 112.8],
  [90.7, 96.6, 88.4, 108.5, 91.1],
  [81.2, 75.1, 75.9, 136.7, 79.0],
  [70.2, 56.3, 70.7, 92.5, 61.7],
  [73.2, 54.0, 82.2, 70.2, 84.9],
  [47.6, 55.2, 76.0, 58.4, 69.5]],
 'PM 1 in ug/m3 (PMS5003_AVG-PM1)': [[14.7, 10.9, 7.9, 10.4, 10.0],
  [16.0, 8.1, 4.5, 10.1, 10.5],
  [24.2, 12.6, 9.8, 9.6, 12.1],
  [9.4, 13.6, 11.4, 9.4, 14.4],
  [11.6, 14.7, 7.1, 17.3, 10.6],
  [11.8, 10.8, 11.7, 20.9, 7.7],
  [13.2, 7.8, 6.3, 18.2, 12.3],
  [9.9, 7.5, 5.0, 18.1, 9.3]],
 'PM 10 in ug/m3 